In [1]:
from __future__ import division

import keras
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
#from keras.utils.visualize_util import plot
from keras.utils import np_utils

from keras.utils.vis_utils import model_to_dot, plot_model
from IPython.display import SVG

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
#keras.backend.image_dim_ordering='th'


from keras import backend as K

Using TensorFlow backend.


In [2]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense

vgg16 = VGG16(weights='imagenet')


In [3]:
# Freeze all layers for now.. 
for layer in vgg16.layers[:15]:
    layer.trainable = False
    
convs = vgg16.get_layer('flatten').output

#create new layers
new = Dense(1024, activation='relu', name='fc6_pose')(convs)
new = Dense(1024, activation='relu', name='fc7_pose')(new)

# predict locations

# output : 2 visibility probabilities
vis1 = Dense(2, activation='softmax', name='visibility1')(new)
vis2 = Dense(2, activation='softmax', name='visibility2')(new)
vis3 = Dense(2, activation='softmax', name='visibility3')(new)
vis4 = Dense(2, activation='softmax', name='visibility4')(new)
vis5 = Dense(2, activation='softmax', name='visibility5')(new)
vis6 = Dense(2, activation='softmax', name='visibility6')(new)

# landmark outputs
lm=Dense(12,activation='linear',name='landmarks')(new)

outputs=[vis1, vis2, vis3, vis4,vis5, vis6, lm]

model = Model(input=vgg16.input, output=outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

C:\Users\1023473\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`


In [4]:
train_index=pd.read_csv('final_csv.csv')
train_index=train_index.set_index('path')
train_index.head()

,image,image_name,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,...,landmark_location_y_3_sld,landmark_location_x_4_sld,landmark_location_y_4_sld,landmark_location_x_5_sld,landmark_location_y_5_sld,landmark_location_x_6_sld,landmark_location_y_6_sld,lms,evaluation_status,file
path,,,,,,,,,,,,,,,,,,,,,
train/img_00000001.jpg,img_00000001.jpg,img/img_00000001.jpg,1,1,0,144,156,0,221,159,...,0.424361,0.490119,0.465483,0.225873,0.792829,0.469636,0.801193,"[0.2993492407809111, 0.3681710213776722, 0.444...",train,train/img_00000001.jpg
val/img_00000002.jpg,img_00000002.jpg,img/img_00000002.jpg,1,3,1,111,104,0,153,105,...,0.282908,0.353755,0.317554,0.170431,0.354582,0.267206,0.375746,"[0.227765726681128, 0.24465558194774348, 0.298...",val,val/img_00000002.jpg
train/img_00000003.jpg,img_00000003.jpg,img/img_00000003.jpg,1,1,1,112,113,0,172,117,...,0.330059,0.411067,0.339250,0.205339,0.531873,0.368421,0.502982,"[0.2299349240780911, 0.2660332541567696, 0.339...",train,train/img_00000003.jpg
train/img_00000004.jpg,img_00000004.jpg,img/img_00000004.jpg,1,2,1,76,74,1,108,69,...,0.220039,0.276680,0.193294,0.133470,0.326693,0.244939,0.320080,"[0.15184381778741865, 0.17339667458432304, 0.2...",train,train/img_00000004.jpg
train/img_00000005.jpg,img_00000005.jpg,img/img_00000005.jpg,1,1,1,217,139,1,290,156,...,0.434185,0.644269,0.489152,0.373717,0.605578,0.615385,0.604374,"[0.45770065075921906, 0.32779097387173395, 0.5...",train,train/img_00000005.jpg


In [5]:
def landmarks(df):
    my_list=[df.landmark_location_x_1_sld,df.landmark_location_y_1_sld,df.landmark_location_x_2_sld,df.landmark_location_y_2_sld,
             df.landmark_location_x_3_sld,df.landmark_location_y_3_sld,df.landmark_location_x_4_sld,df.landmark_location_y_4_sld,
             df.landmark_location_x_5_sld,df.landmark_location_y_5_sld,df.landmark_location_x_6_sld,df.landmark_location_y_6_sld]
    return my_list

train_index['lms']=train_index.apply(landmarks,axis=1)


Y1=train_index[['landmark_visibility_1']]
Y2=train_index[['landmark_visibility_2']]
Y3=train_index[['landmark_visibility_3']]
Y4=train_index[['landmark_visibility_4']]
Y5=train_index[['landmark_visibility_5']]
Y6=train_index[['landmark_visibility_6']]

In [6]:
from keras.preprocessing import image
import keras.backend as K

train_gen= keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None)

In [7]:
# limiting batch size to 16 ~ due to GPU mem issues
train_generator = train_gen.flow_from_directory(
        'C:/Users/1023473/Downloads/img-002/img/train',
        target_size=(224, 224),
        batch_size=16,
        class_mode='sparse')

Found 0 images belonging to 83033 classes.
